<a href="https://colab.research.google.com/github/2019mohamed/seq2seq_constrains_dates/blob/main/seq2seq_constrains_dates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchtext==0.6.0 --quiet

     |████████████████████████████████| 64 kB 3.1 MB/s 
     |████████████████████████████████| 1.2 MB 65.3 MB/s 


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator, TabularDataset
import numpy as np
import pandas as pd
import spacy
import random
from torchtext.data.metrics import bleu_score
from pprint import pprint
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

#Reading dataset 

In [4]:
data = open('/content/drive/MyDrive/data.txt').read().split('\n')

In [5]:
data

['[WED] [JAN] [False] [180] 1-1-1800',
 '[THU] [JAN] [False] [180] 1-1-1801',
 '[FRI] [JAN] [False] [180] 1-1-1802',
 '[SAT] [JAN] [False] [180] 1-1-1803',
 '[SUN] [JAN] [True] [180] 1-1-1804',
 '[TUE] [JAN] [False] [180] 1-1-1805',
 '[WED] [JAN] [False] [180] 1-1-1806',
 '[THU] [JAN] [False] [180] 1-1-1807',
 '[FRI] [JAN] [True] [180] 1-1-1808',
 '[SUN] [JAN] [False] [180] 1-1-1809',
 '[MON] [JAN] [False] [181] 1-1-1810',
 '[TUE] [JAN] [False] [181] 1-1-1811',
 '[WED] [JAN] [True] [181] 1-1-1812',
 '[FRI] [JAN] [False] [181] 1-1-1813',
 '[SAT] [JAN] [False] [181] 1-1-1814',
 '[SUN] [JAN] [False] [181] 1-1-1815',
 '[MON] [JAN] [True] [181] 1-1-1816',
 '[WED] [JAN] [False] [181] 1-1-1817',
 '[THU] [JAN] [False] [181] 1-1-1818',
 '[FRI] [JAN] [False] [181] 1-1-1819',
 '[SAT] [JAN] [True] [182] 1-1-1820',
 '[MON] [JAN] [False] [182] 1-1-1821',
 '[TUE] [JAN] [False] [182] 1-1-1822',
 '[WED] [JAN] [False] [182] 1-1-1823',
 '[THU] [JAN] [True] [182] 1-1-1824',
 '[SAT] [JAN] [False] [182] 1-1

# Preparing dataset
1-split constrains and date.

2-tokenization.

3-preparing train dataset.

In [6]:
!python -m spacy download en --quiet

     |████████████████████████████████| 12.0 MB 29.9 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [7]:
nlp = spacy.load('en_core_web_sm')

In [8]:
out = data[0].split()

In [9]:
splited_data = [d.split() for d in data]

In [10]:
splited_data

[['[WED]', '[JAN]', '[False]', '[180]', '1-1-1800'],
 ['[THU]', '[JAN]', '[False]', '[180]', '1-1-1801'],
 ['[FRI]', '[JAN]', '[False]', '[180]', '1-1-1802'],
 ['[SAT]', '[JAN]', '[False]', '[180]', '1-1-1803'],
 ['[SUN]', '[JAN]', '[True]', '[180]', '1-1-1804'],
 ['[TUE]', '[JAN]', '[False]', '[180]', '1-1-1805'],
 ['[WED]', '[JAN]', '[False]', '[180]', '1-1-1806'],
 ['[THU]', '[JAN]', '[False]', '[180]', '1-1-1807'],
 ['[FRI]', '[JAN]', '[True]', '[180]', '1-1-1808'],
 ['[SUN]', '[JAN]', '[False]', '[180]', '1-1-1809'],
 ['[MON]', '[JAN]', '[False]', '[181]', '1-1-1810'],
 ['[TUE]', '[JAN]', '[False]', '[181]', '1-1-1811'],
 ['[WED]', '[JAN]', '[True]', '[181]', '1-1-1812'],
 ['[FRI]', '[JAN]', '[False]', '[181]', '1-1-1813'],
 ['[SAT]', '[JAN]', '[False]', '[181]', '1-1-1814'],
 ['[SUN]', '[JAN]', '[False]', '[181]', '1-1-1815'],
 ['[MON]', '[JAN]', '[True]', '[181]', '1-1-1816'],
 ['[WED]', '[JAN]', '[False]', '[181]', '1-1-1817'],
 ['[THU]', '[JAN]', '[False]', '[181]', '1-1-1818'

In [11]:
dates = [d[-1] for d in splited_data]
const = [' '.join(d[:4]) for d in splited_data]

In [12]:
dates

['1-1-1800',
 '1-1-1801',
 '1-1-1802',
 '1-1-1803',
 '1-1-1804',
 '1-1-1805',
 '1-1-1806',
 '1-1-1807',
 '1-1-1808',
 '1-1-1809',
 '1-1-1810',
 '1-1-1811',
 '1-1-1812',
 '1-1-1813',
 '1-1-1814',
 '1-1-1815',
 '1-1-1816',
 '1-1-1817',
 '1-1-1818',
 '1-1-1819',
 '1-1-1820',
 '1-1-1821',
 '1-1-1822',
 '1-1-1823',
 '1-1-1824',
 '1-1-1825',
 '1-1-1826',
 '1-1-1827',
 '1-1-1828',
 '1-1-1829',
 '1-1-1830',
 '1-1-1831',
 '1-1-1832',
 '1-1-1833',
 '1-1-1834',
 '1-1-1835',
 '1-1-1836',
 '1-1-1837',
 '1-1-1838',
 '1-1-1839',
 '1-1-1840',
 '1-1-1841',
 '1-1-1842',
 '1-1-1843',
 '1-1-1844',
 '1-1-1845',
 '1-1-1846',
 '1-1-1847',
 '1-1-1848',
 '1-1-1849',
 '1-1-1850',
 '1-1-1851',
 '1-1-1852',
 '1-1-1853',
 '1-1-1854',
 '1-1-1855',
 '1-1-1856',
 '1-1-1857',
 '1-1-1858',
 '1-1-1859',
 '1-1-1860',
 '1-1-1861',
 '1-1-1862',
 '1-1-1863',
 '1-1-1864',
 '1-1-1865',
 '1-1-1866',
 '1-1-1867',
 '1-1-1868',
 '1-1-1869',
 '1-1-1870',
 '1-1-1871',
 '1-1-1872',
 '1-1-1873',
 '1-1-1874',
 '1-1-1875',
 '1-1-1876',

In [13]:
const

['[WED] [JAN] [False] [180]',
 '[THU] [JAN] [False] [180]',
 '[FRI] [JAN] [False] [180]',
 '[SAT] [JAN] [False] [180]',
 '[SUN] [JAN] [True] [180]',
 '[TUE] [JAN] [False] [180]',
 '[WED] [JAN] [False] [180]',
 '[THU] [JAN] [False] [180]',
 '[FRI] [JAN] [True] [180]',
 '[SUN] [JAN] [False] [180]',
 '[MON] [JAN] [False] [181]',
 '[TUE] [JAN] [False] [181]',
 '[WED] [JAN] [True] [181]',
 '[FRI] [JAN] [False] [181]',
 '[SAT] [JAN] [False] [181]',
 '[SUN] [JAN] [False] [181]',
 '[MON] [JAN] [True] [181]',
 '[WED] [JAN] [False] [181]',
 '[THU] [JAN] [False] [181]',
 '[FRI] [JAN] [False] [181]',
 '[SAT] [JAN] [True] [182]',
 '[MON] [JAN] [False] [182]',
 '[TUE] [JAN] [False] [182]',
 '[WED] [JAN] [False] [182]',
 '[THU] [JAN] [True] [182]',
 '[SAT] [JAN] [False] [182]',
 '[SUN] [JAN] [False] [182]',
 '[MON] [JAN] [False] [182]',
 '[TUE] [JAN] [True] [182]',
 '[THU] [JAN] [False] [182]',
 '[FRI] [JAN] [False] [183]',
 '[SAT] [JAN] [False] [183]',
 '[SUN] [JAN] [True] [183]',
 '[TUE] [JAN] [Fal

In [14]:
raw_data = {'const':const , 'dates': dates }

In [15]:
import pandas as pd

In [16]:
df = pd.DataFrame(raw_data , columns= ['const', 'dates'])
df.head()

,const,dates
0,[WED] [JAN] [False] [180],1-1-1800
1,[THU] [JAN] [False] [180],1-1-1801
2,[FRI] [JAN] [False] [180],1-1-1802
3,[SAT] [JAN] [False] [180],1-1-1803
4,[SUN] [JAN] [True] [180],1-1-1804


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146462 entries, 0 to 146461
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   const   146462 non-null  object
 1   dates   146462 non-null  object
dtypes: object(2)
memory usage: 2.2+ MB


In [18]:
from sklearn.model_selection import train_test_split

train , test = train_test_split(df , test_size=0.1)

In [19]:
train.to_json('train.json', orient="records",lines=True)
test.to_json('test.json',orient= "records",lines=True)

In [20]:
def tokenzie_date (date):
  return date.split('-')

tokenzie_date("1-1-1800")

['1', '1', '1800']

In [21]:
def tokenzie_const (const):
  out =  [t.text for t in nlp.tokenizer(const)]
  return list(filter(lambda x : x not in ['[' , ']'] , out))

tokenzie_const('[SUN] [JAN] [True] [180]')

['SUN', 'JAN', 'True', '180']

# torchtext 
torchtext's `Field`s handle how data should be processed. All of the possible arguments are detailed [here](https://github.com/pytorch/text/blob/master/torchtext/data/field.py#L61). 

We set the `tokenize` argument to the correct tokenization function for each, with German being the `SRC` (source) field and English being the `TRG` (target) field. The field also appends the "start of sequence" and "end of sequence" tokens via the `init_token` and `eos_token` arguments, and converts all words to lowercase.

In [22]:
Fconst = Field(sequential = True,use_vocab = True,tokenize = tokenzie_const ,lower = True,init_token="<sos>",
               eos_token="<eos>")
Fdate = Field(sequential = True,use_vocab = True,tokenize = tokenzie_date ,lower = True,init_token="<sos>",
               eos_token="<eos>")

In [23]:
fields={'const':('const',Fconst),  'dates': ('dates',Fdate)}

In [24]:
train_data,test_data =  TabularDataset.splits(
   path='',train='train.json',test="test.json",format='json',fields=fields)

In [25]:
Fconst.build_vocab(train_data, max_size= 1000,min_freq=0)
Fdate.build_vocab(train_data, max_size = 1000,min_freq=0)

In [26]:
train_iterator,test_iterator=BucketIterator.splits(
    (train_data,test_data),
    batch_size=512, device = device)

In [27]:
len(Fconst.vocab) , ' ',len(Fdate.vocab)

(66, ' ', 436)

In [28]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 131815
Number of testing examples: 14647


In [29]:
print(Fconst.vocab.__dict__.keys())
print(list(Fconst.vocab.__dict__.values()))
print(list(Fdate.vocab.__dict__.values()))

dict_keys(['freqs', 'itos', 'unk_index', 'stoi', 'vectors'])
[Counter({'false': 99837, 'true': 31978, 'wed': 18860, 'sun': 18854, 'tue': 18846, 'thu': 18834, 'sat': 18817, 'fri': 18806, 'mon': 18798, 'oct': 11225, 'mar': 11218, 'jul': 11196, 'may': 11192, 'aug': 11187, 'jan': 11182, 'dec': 11146, 'apr': 10904, 'jun': 10844, 'nov': 10826, 'sep': 10779, 'feb': 10116, '184': 3331, '182': 3329, '201': 3315, '208': 3311, '200': 3308, '212': 3306, '203': 3306, '211': 3303, '207': 3299, '183': 3297, '218': 3296, '204': 3295, '186': 3294, '196': 3294, '181': 3292, '210': 3292, '214': 3292, '188': 3292, '216': 3291, '205': 3289, '219': 3287, '190': 3285, '192': 3284, '202': 3283, '194': 3283, '213': 3282, '185': 3279, '195': 3279, '180': 3279, '193': 3278, '206': 3275, '198': 3272, '197': 3267, '215': 3266, '217': 3265, '189': 3261, '199': 3260, '187': 3260, '209': 3259, '191': 3256, '220': 323}), ['<unk>', '<pad>', '<sos>', '<eos>', 'false', 'true', 'wed', 'sun', 'tue', 'thu', 'sat', 'fri', 'm

In [30]:
for data in train_iterator:
  print(data.const,' ',data.dates)

tensor([[ 2,  2,  2,  ...,  2,  2,  2],
        [ 6, 10,  7,  ...,  9, 10, 12],
        [21, 18, 16,  ..., 15, 18, 18],
        [ 4,  4,  4,  ...,  4,  5,  5],
        [47, 63, 50,  ..., 65, 27, 59],
        [ 3,  3,  3,  ...,  3,  3,  3]], device='cuda:0')   tensor([[  2,   2,   2,  ...,   2,   2,   2],
        [ 23,   8,  17,  ...,  19,   6,  16],
        [ 12,   9,   8,  ...,   6,   9,   9],
        [ 55, 358, 360,  ..., 376,  89, 252],
        [  3,   3,   3,  ...,   3,   3,   3]], device='cuda:0')
tensor([[ 2,  2,  2,  ...,  2,  2,  2],
        [ 9,  7,  8,  ...,  9,  6,  9],
        [14, 23, 17,  ..., 14, 24, 16],
        [ 5,  4,  4,  ...,  4,  4,  5],
        [34, 41, 46,  ..., 26, 48, 62],
        [ 3,  3,  3,  ...,  3,  3,  3]], device='cuda:0')   tensor([[  2,   2,   2,  ...,   2,   2,   2],
        [ 29,  16,   9,  ...,  33,  23,  20],
        [  5,  14,   7,  ...,   5,  15,   8],
        [185, 159, 363,  ...,  79, 260, 128],
        [  3,   3,   3,  ...,   3,   3,   3]], d

## Building the Seq2Seq Model

The model is used in https://github.com/bentrevett/pytorch-seq2seq. For simplicity, we used LSTM encoder and decoder. We can use transformer as encoder decoder to enhance the output sequance(i.e., dates)

In [31]:
class EncoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
    super(EncoderLSTM, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    #self.input_size = input_size

    # Output size of the word embedding NN
    #self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Regularization parameter
    self.dropout = nn.Dropout(p)
    self.tag = True

    # Shape --------------------> (5376, 300) [input size, embedding dims]
    self.embedding = nn.Embedding(input_size, embedding_size)
    
    # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

  # Shape of x (26, 32) [Sequence_length, batch_size]
  def forward(self, x):

    # Shape -----------> (26, 32, 300) [Sequence_length , batch_size , embedding dims]
    embedding = self.dropout(self.embedding(x))
    
    # Shape --> outputs (26, 32, 1024) [Sequence_length , batch_size , hidden_size]
    # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size]
    outputs, (hidden_state, cell_state) = self.LSTM(embedding)

    return hidden_state, cell_state

input_size_encoder = len(Fconst.vocab)
encoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
encoder_dropout = 0.5

encoder_lstm = EncoderLSTM(input_size_encoder, encoder_embedding_size,
                           hidden_size, num_layers, encoder_dropout).to(device)
print(encoder_lstm)

EncoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(66, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
)


In [32]:
class DecoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p, output_size):
    super(DecoderLSTM, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    #self.input_size = input_size

    # Output size of the word embedding NN
    #self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Size of the one hot vectors that will be the output to the encoder (English Vocab Size)
    self.output_size = output_size

    # Regularization parameter
    self.dropout = nn.Dropout(p)

    # Shape --------------------> (5376, 300) [input size, embedding dims]
    self.embedding = nn.Embedding(input_size, embedding_size)

    # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

    # Shape -----------> (1024, 4556) [embedding dims, hidden size, num layers]
    self.fc = nn.Linear(hidden_size, output_size)

  # Shape of x (32) [batch_size]
  def forward(self, x, hidden_state, cell_state):

    # Shape of x (1, 32) [1, batch_size]
    x = x.unsqueeze(0)

    # Shape -----------> (1, 32, 300) [1, batch_size, embedding dims]
    embedding = self.dropout(self.embedding(x))

    # Shape --> outputs (1, 32, 1024) [1, batch_size , hidden_size]
    # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size] (passing encoder's hs, cs - context vectors)
    outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))

    # Shape --> predictions (1, 32, 4556) [ 1, batch_size , output_size]
    predictions = self.fc(outputs)

    # Shape --> predictions (32, 4556) [batch_size , output_size]
    predictions = predictions.squeeze(0)

    return predictions, hidden_state, cell_state

input_size_decoder = len(Fdate.vocab)
decoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
decoder_dropout = 0.5
output_size = len(Fdate.vocab)

decoder_lstm = DecoderLSTM(input_size_decoder, decoder_embedding_size,
                           hidden_size, num_layers, decoder_dropout, output_size).to(device)
print(decoder_lstm)

DecoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(436, 300)
  (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  (fc): Linear(in_features=1024, out_features=436, bias=True)
)


In [33]:
for batch in train_iterator:
  print(batch.const.shape)
  print(batch.dates.shape)
  break

x = batch.const[1]
print(x)

torch.Size([6, 512])
torch.Size([5, 512])
tensor([ 8, 12, 10,  6,  6,  6,  7, 11, 11, 10,  9, 12, 12,  9,  6,  7, 11,  9,
         6,  9,  6,  9, 12, 10,  7,  6,  7, 12,  9,  9,  9,  8,  7,  9,  9, 10,
         9,  6, 12, 10,  6,  6, 10, 11,  8,  7, 12, 10,  7, 11,  9, 11,  9, 11,
         8,  6,  8, 12, 12,  7, 12,  9, 10, 11, 12,  8,  9,  6, 10, 10, 10, 11,
         6,  7, 12,  6, 10, 12,  8, 10, 10,  8, 11,  9,  6,  9, 10,  8,  7, 11,
        12,  7, 12, 10, 11,  6, 10, 10, 10, 10, 12,  8,  7, 11,  6, 12, 12, 10,
         9, 10,  9,  7,  8,  6,  7, 10, 12,  6,  8, 12, 11,  8,  9,  7, 10, 12,
         6, 11,  6,  7, 12,  7,  9,  7,  8, 12,  9, 11,  9, 12,  7,  6,  9, 12,
        10, 10,  9, 12, 10, 10, 10,  8,  8, 10, 11,  8, 10,  7,  7, 11,  8,  6,
         8,  8, 10,  9,  9,  6, 11, 12,  9, 12, 10,  7, 12, 11,  7,  8,  6,  9,
         7,  9,  7, 11,  8, 12,  7, 12,  6, 11, 11,  7, 11, 11, 10, 12,  7, 12,
        12,  8,  7,  6,  8, 11,  6, 10, 10, 11,  9,  9, 12,  9, 10,  7, 12,  7

### Seq2Seq

For the final part of the implemenetation, we'll implement the seq2seq model. This will handle: 
- receiving the input/source sentence
- using the encoder to produce the context vectors 
- using the decoder to produce the predicted output/target sentence

In [34]:
class Seq2Seq(nn.Module):
  def __init__(self, Encoder_LSTM, Decoder_LSTM):
    super(Seq2Seq, self).__init__()
    self.Encoder_LSTM = Encoder_LSTM
    self.Decoder_LSTM = Decoder_LSTM

  def forward(self, source, target, tfr=0.5):
    # Shape - Source : (10, 32) [(Sentence length German + some padding), Number of Sentences]
    batch_size = source.shape[1]

    # Shape - Source : (14, 32) [(Sentence length English + some padding), Number of Sentences]
    target_len = target.shape[0]
    target_vocab_size = len(Fdate.vocab)
    
    # Shape --> outputs (14, 32, 5766) 
    outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

    # Shape --> (hs, cs) (2, 32, 1024) ,(2, 32, 1024) [num_layers, batch_size size, hidden_size] (contains encoder's hs, cs - context vectors)
    hidden_state, cell_state = self.Encoder_LSTM(source)

    # Shape of x (32 elements)
    x = target[0] # Trigger token <SOS>

    for i in range(1, target_len):
      # Shape --> output (32, 5766) 
      output, hidden_state, cell_state = self.Decoder_LSTM(x, hidden_state, cell_state)
      outputs[i] = output
      best_guess = output.argmax(1) # 0th dimension is batch size, 1st dimension is word embedding
      x = target[i] if random.random() < tfr else best_guess # Either pass the next word correctly from the dataset or use the earlier predicted word

    # Shape --> outputs (14, 32, 5766) 
    return outputs

In [35]:
# Hyperparameters

learning_rate = 0.001
writer = SummaryWriter(f"runs/loss_plot")
step = 0

model = Seq2Seq(encoder_lstm, decoder_lstm).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

pad_idx = Fdate.vocab.stoi["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [36]:
model

Seq2Seq(
  (Encoder_LSTM): EncoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(66, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
  )
  (Decoder_LSTM): DecoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(436, 300)
    (LSTM): LSTM(300, 1024, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=1024, out_features=436, bias=True)
  )
)

In [37]:
Fconst.eos_token

'<eos>'

In [38]:
def translate_sentence(model, sentence, Fconst, Fdate, max_length=3):

    tokens = tokenzie_const(sentence)
    tokens.insert(0, Fconst.init_token)
    tokens.append(Fconst.eos_token)
    
    text_to_indices = [Fconst.vocab.stoi[token] for token in tokens]
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        hidden, cell = model.Encoder_LSTM(sentence_tensor)

    outputs = [Fdate.vocab.stoi["<sos>"]]
    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.Decoder_LSTM(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == Fdate.vocab.stoi["<eos>"]:
            break

    translated_sentence = [Fdate.vocab.itos[idx] for idx in outputs]
    return translated_sentence[1:]


In [39]:
translate_sentence(model, "[TUE] [JUN] [False] [189]", Fconst, Fdate)

['1911', '2198', '1871']

# Training the Seq2Seq Model


In [41]:
epoch_loss = 0.0
num_epochs = 5
best_loss = 999999
best_epoch = -1
sentence1 = "[TUE] [JUN] [False] [189]" #"21-6-1898"
ts1  = []

for epoch in range(num_epochs):
  print("Epoch - {} / {}".format(epoch+1, num_epochs))
  model.eval()
  translated_sentence1 = translate_sentence(model, sentence1, Fconst, Fdate)
  print(f"Translated example sentence 1: \n {translated_sentence1}")
  ts1.append(translated_sentence1)

  model.train(True)
  for batch_idx, batch in enumerate(train_iterator):
    input = batch.const.to(device)
    target = batch.dates.to(device)

    # Pass the input and target for model's forward method
    output = model(input, target)
    output = output[1:].reshape(-1, output.shape[2])
    target = target[1:].reshape(-1)

    # Clear the accumulating gradients
    optimizer.zero_grad()

    # Calculate the loss value for every epoch
    loss = criterion(output, target)

    # Calculate the gradients for weights & biases using back-propagation
    loss.backward()

    # Clip the gradient value is it exceeds > 1
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

    # Update the weights values using the gradients we calculated using bp 
    optimizer.step()
    step += 1
    epoch_loss += loss.item()
    writer.add_scalar("Training loss", loss, global_step=step)

  if epoch_loss < best_loss:
    best_loss = epoch_loss
    best_epoch = epoch
    torch.save(model.state_dict(), 'model.pth')
    if ((epoch - best_epoch) >= 10):
      print("no improvement in 10 epochs, break")
      break
  print("Epoch_Loss - {}".format(loss.item()))
  print()
  
print(epoch_loss / len(train_iterator))


Epoch - 1 / 5
Translated example sentence 1: 
 ['7', '12', '1895']
Epoch_Loss - 1.3264899253845215

Epoch - 2 / 5
Translated example sentence 1: 
 ['28', '11', '1890']
Epoch_Loss - 1.3204458951950073

Epoch - 3 / 5
Translated example sentence 1: 
 ['19', '10', '1890']
Epoch_Loss - 1.3082034587860107

Epoch - 4 / 5
Translated example sentence 1: 
 ['16', '12', '1898']
Epoch_Loss - 1.310160517692566

Epoch - 5 / 5
Translated example sentence 1: 
 ['20', '11', '1894']
Epoch_Loss - 1.3125947713851929

6.606837358585624


#Loading model

In [42]:
model.load_state_dict(torch.load('/content/model.pth'))

<All keys matched successfully>

In [43]:
translate_sentence(model , sentence1 , Fconst , Fdate)

['28', '11', '1894']

In [44]:
test 

,const,dates
67676,[SUN] [JAN] [True] [210],15-1-2108
87449,[SAT] [MAR] [False] [183],19-3-1831
50442,[FRI] [JUN] [False] [211],11-6-2117
15793,[SUN] [APR] [False] [195],4-4-1954
37482,[SUN] [OCT] [False] [198],8-10-1989
...,...,...
91434,[MON] [JAN] [False] [180],20-1-1806
123262,[FRI] [AUG] [False] [195],26-8-1955
52903,[FRI] [DEC] [True] [217],11-12-2172
7981,[MON] [AUG] [False] [216],2-8-2162


In [45]:
list_const = test['const'].tolist()
list_date = test['dates'].tolist()

#Evaluating model

In [46]:
def is_leap_year(year):
  return ((year % 100 != 0) and (year % 4 == 0)) or (year % 400 == 0)

In [47]:
year_index = {'JAN':1 , 'FEB':2, 'MAR':3 , 'APR':4 , 'MAY':5 , 'JUN':6 , 'JUL':7 , 'AUG':8,'SEP':9,'OCT':10,'NOV':11,'DEC':12}

In [48]:
from datetime import datetime

format = '%d-%m-%Y'
def valid_date (date):
  res = True
  try:
    res = bool(datetime.strptime(date , format))
  except ValueError:
    res = False
  return res

In [49]:
day_name= ['MON', 'TUE', 'WED', 'THU', 'FRI', 'SAT','SUN']
day = datetime.strptime('16-3-2100', format).weekday()
day_name[day]

'TUE'

### Our method for evaluating 

    if constrains contains [True] :
      if satisfy one of the constrains:
        acc += 0.25
    else:
      if satisfy one of the constrains:
        acc += 1/3


In [50]:
acc = 0
for cons , date in zip(list_const , list_date):
  predict_date = translate_sentence(model , cons , Fconst , Fdate)
  predict_date_str = '-'.join(predict_date)
  if valid_date(predict_date_str):
    toc_cons = tokenzie_const(cons)
    toc_date = tokenzie_date(date)
    if toc_cons[2]:
      if is_leap_year(int(predict_date[2])):
        acc += .25
      if int(predict_date[2])/10 == int(toc_cons[3]):
        acc += .25
      if int(predict_date[1]) == int(toc_date[1]):
        acc += .25
      if datetime.strptime(predict_date_str, format).weekday() == toc_cons[0]:
        acc += .25
      
    else:
      if int(predict_date[2])/10 == int(toc_cons[3]):
        acc += 1/3
      if int(predict_date[1]) == int(toc_date[1]):
        acc += 1/3
      if datetime.strptime(predict_date_str, format).weekday() == toc_cons[0]:
        acc += 1/3

acc /= len(test)

In [51]:
acc * 100.0 # = 4 when ephoc = 100%

5.53697002799208